In [ ]:
import os
import tifffile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from natsort import natsorted
from matplotlib.colors import Normalize

In [ ]:
# Set the input directory path
input_dir = "E:/Temperature_images/RAW_2023-07-13/"
output_path = "E:/Temperature_images/RAW_2023-07-13/Stacks/"

# Create the output directory if it doesn't already exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Get a list of all TIFF files in the input directory
input_files = [f for f in os.listdir(input_dir) if f.endswith('.tiff')]
input_files = natsorted(input_files)
# Reshape array of input files
files_list = np.reshape(input_files, (-1, 1))
print(files_list)
# Initialize empty lists for the red and green stacks
red_stack = []
green_stack = []
blue_stack = []
raw_stack =[]

# Loop through each image file in the input directory
for file in input_files:
    # Load the image using tifffile
    img = tifffile.imread(os.path.join(input_dir, file))

    ### Set Crop dimension (OPTIONAL) ###
    img = img[1500:3000,0:1800] 
    
    # Extract the red and green channels
    red_channel = img[1::2, 1::2]
    blue_channel = img[0::2, 0::2]
    green_channel_1 = img[0::2, 1::2]
    green_channel_2 = img[1::2, 0::2]
    green_channel = np.add(green_channel_1, green_channel_2) // 2
     # Add the red and green channels to the corresponding stacks
    blue_stack.append(blue_channel)
    red_stack.append(red_channel)
    green_stack.append(green_channel)
    raw_stack.append(img)

# Stack the red and green channel lists along the third axis to create the corresponding stacks
red_stack = np.stack(red_stack, axis=0)
green_stack = np.stack(green_stack, axis=0)
blue_stack =np.stack(blue_stack, axis=0)

# Save the red and green stacks to separate TIFF files
tifffile.imwrite(os.path.join(output_path, "stacked_red_channel.tiff"), red_stack)
tifffile.imwrite(os.path.join(output_path, "stacked_green_channel.tiff"), green_stack)
tifffile.imwrite(os.path.join(output_path, "stacked_blue_channel.tiff"), blue_stack)




In [ ]:

# Define the measurement from the Soil_Images folder
Folder = "Temperature_images"
Mearsurements = "RAW_2023-07-13"

# Set the path to the directory containing the image stacks
stack_dir = f'E:/{Folder}/{Mearsurements}/Stacks'


# Get a list of all TIFF files in the input directory
files = [f for f in os.listdir(stack_dir) if f.endswith('.tiff')]
print(files)

green_stack = tifffile.imread(os.path.join(stack_dir, files[1]))
red_stack = tifffile.imread(os.path.join(stack_dir, files[2]))
blue_stack = tifffile.imread(os.path.join(stack_dir, files[0]))

#Get length of image stacks
num_images_green = len(green_stack)
num_images_red = len(red_stack)
num_images_blue = len(blue_stack)
if num_images_green == num_images_red and num_images_blue:
    print("Equal stacks")    
else:
    print("unequal stacks")

num_images = num_images_green
print("images in stacks=",num_images)

In [ ]:
import tifffile
import os

def extract_and_save_images(tiff_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the TIFF file
    with tifffile.TiffFile(tiff_path) as tif:
        # Iterate over each frame in the TIFF stack
        for i, page in enumerate(tif.pages):
            image = page.asarray()  # Convert the current frame to a numpy array
            # Save the image
            output_path = os.path.join(output_folder, f'image_{i+1}.tiff')
            tifffile.imwrite(output_path, image)
            print(f'Saved {output_path}')

# Usage example:
tiff_path = "E:/Temperature_images/RAW_2023-07-13/Stacks/stacked_red_channel.tiff"
output_folder = 'E:/Temperature_images/RAW_2023-07-13/Red_channel'
extract_and_save_images(tiff_path, output_folder)


In [ ]:
# Set the path to the directory containing the ratio images
ratio_img_dir = 'E:/Temperature_images/RAW_2023-07-13/Red_channel/'

output_path = 'E:/Temperature_images/RAW_2023-07-13/Red_channel_calibrated/'

# Create the output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Get a list of all TIFF files in the input directory and sort them naturally
files = natsorted([f for f in os.listdir(ratio_img_dir) if f.endswith('.tiff')])
print(files)

# SET a and b FROM CALIBRATION
b = 2284.04
a = 40.67

def calculate_x(y, b, a):
    x = (y-b)/-a
    return x


# Calculate and save the images
for i, file in enumerate(files):
    ratio_img = tifffile.imread(os.path.join(ratio_img_dir, file))

    red_blue_ratio = ratio_img

    # Calculate Temperature for each pixel
    temperature = calculate_x(red_blue_ratio, b, a)

    # Construct the output file path
    output_file = os.path.join(output_path, f"ratio_image_calibrated_{i}.tiff")

    # Save the temperature image using imwrite
    tifffile.imwrite(output_file, temperature.astype(np.float32))

print("Processing complete.")

In [ ]:
# Set the path to the directory containing the calibrated images
calibrated_img_dir = 'E:/Temperature_images/RAW_2023-07-13/Red_channel_calibrated/'

# Get a list of all TIFF files in the calibrated images directory and sort them naturally
files = natsorted([f for f in os.listdir(calibrated_img_dir) if f.endswith('.tiff')])


# Display the normalized images using matplotlib
for i, file in enumerate(files):
    calibrated_image = tifffile.imread(os.path.join(calibrated_img_dir, file))

    # Flip the image by 180 degrees using numpy
    flipped_image = np.flipud(np.fliplr(calibrated_image))

    plt.imshow(flipped_image, cmap='inferno',vmin=0,vmax=25)
    plt.title(f"Calibrated Image {i}")
    plt.colorbar(label='O2 Concentration (%)')
    plt.show()

In [ ]:
import os
import tifffile
import matplotlib.pyplot as plt
from natsort import natsorted

# Set the path to the directory containing the calibrated images
calibrated_img_dir = f'E:/{Folder}/{Mearsurements}/Ratio_Images_Calibrated/'

# Get a list of all TIFF files in the calibrated images directory and sort them naturally
files = natsorted([f for f in os.listdir(calibrated_img_dir) if f.endswith('.tiff')])

# Directory to save the images with the specified colormap range as TIFF files
output_dir = f'E:/{Folder}/{Mearsurements}/Images_Calibrated/'
os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist

# Define the desired colormap range
colormap_min = 0  # Set your desired minimum value
colormap_max = 25 # Set your desired maximum value

for i, file in enumerate(files):
    calibrated_image = tifffile.imread(os.path.join(calibrated_img_dir, file))

    # Apply the colormap with the specified range
    cmap = plt.get_cmap('inferno')
    normed_image = cmap((calibrated_image - colormap_min) / (colormap_max - colormap_min))

    # Convert the image to uint8 (8-bit) data type
    normed_image_uint8 = (normed_image * 255.0).astype('uint8')

    # Save the image as a TIFF file with the specified colormap range and uint8 data type
    output_filename = os.path.join(output_dir, f"calibrated_image_{i}.tiff")
    tifffile.imwrite(output_filename, normed_image_uint8)

print("Images saved as TIFF with uint8 data type to:", output_dir)

In [ ]:
# Set the path to the directory containing the calibrated images
calibrated_img_dir = f'E:/{Folder}/{Mearsurements}/Ratio_Images_Calibrated/'

# Get a list of all TIFF files in the calibrated images directory and sort them naturally
files = natsorted([f for f in os.listdir(calibrated_img_dir) if f.endswith('.tiff')])

# Create a figure with subplots to display the images
num_images = len(files)
rows = int(np.sqrt(num_images))
cols = int(np.ceil(num_images / rows))

plt.figure(figsize=(15, 15))

for i, file in enumerate(files):
    calibrated_image = tifffile.imread(os.path.join(calibrated_img_dir, file))

    # Flip the image by 180 degrees using numpy
    flipped_image = np.flipud(np.fliplr(calibrated_image))

    # Calculate the average pixel value (assuming it's already in the range 0-25)
    avg_pixel_value = np.mean(flipped_image)

    # Display the image
    ax = plt.subplot(rows, cols, i + 1)
    plt.imshow(flipped_image, cmap='inferno', vmin=0, vmax=25)
    plt.colorbar(label='Temperature')

    # Display the average value on the image
    plt.text(0.05, 0.95, f'{avg_pixel_value:.1f}', color='white', 
             fontsize=12, ha='left', va='top', transform=ax.transAxes)

plt.tight_layout()
plt.show()

In [ ]:
# Set the path to the directory containing the calibrated images
calibrated_img_dir = f'E:/{Folder}/{Mearsurements}/selected/'

# Get a list of all TIFF files in the calibrated images directory and sort them naturally
files = natsorted([f for f in os.listdir(calibrated_img_dir) if f.endswith('.tiff')])

# Create a list of reference temperatures
reference_temperatures = [6.2, 11.1, 15, 19.3]  # Modify this list based on measured reference temp.

# Create a figure with subplots to display the images
num_images = len(files)
rows = 2  # Number of rows
cols = 2  # Number of columns

fig, axs = plt.subplots(rows, cols, figsize=(cols * 8, rows * 8), layout="compressed")

for i, file in enumerate(files):
    calibrated_image = tifffile.imread(os.path.join(calibrated_img_dir, file))

    # Flip the image by 180 degrees using numpy
    flipped_image = np.flipud(np.fliplr(calibrated_image))

    # Calculate the average pixel value
    avg_pixel_value = np.mean(flipped_image)

    # Calculate the row and column indices for the current image
    row_idx = i // cols
    col_idx = i % cols

    # Display the image
    ax = axs[row_idx, col_idx]
    im = ax.imshow(calibrated_image, cmap='inferno', vmin=0, vmax=25)

    # Display the average value and reference temperature on the image
    ref_temp = reference_temperatures[i]
    ax.text(0.02, 0.95, f"Average Temp: {avg_pixel_value:.1f} °C\nReference Temp: {ref_temp} °C", 
            color='white', weight="bold", fontsize=30, ha='left', va='top', transform=ax.transAxes, 
            bbox=dict(facecolor='black', alpha=0.5, edgecolor='none'))

# Add a single colorbar on the right side
cbar = fig.colorbar(im, ax=axs)
cbar.set_label('Temperature °C', fontsize=50)
cbar.ax.tick_params(labelsize=50)

# Save and display the figure
plt.savefig(f'E:/{Folder}/{Mearsurements}/selected//output_figure.png', dpi=300, bbox_inches='tight')
plt.show()
